In [1]:
import tensorflow as tf
import random

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
#Onehot vectoring
mnist = input_data.read_data_sets("MNIST_data", one_hot = True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
#hyper parameters
learning_rate = 0.001
training_epochs = 12
batch_size = 32

In [5]:
#droout_probability
keep_prob = tf.placeholder(tf.float32)

#Placeholder
X = tf.placeholder(tf.float32,[None,784])
X_img = tf.reshape(X, [-1,28,28,1])
Y= tf.placeholder(tf.float32, [None, 10])

In [6]:
#L1 (N, 28,28,1)
W1 = tf.Variable(tf.random_normal([3,3,1,32], stddev = 0.01))
L1 = tf.nn.conv2d(X_img, W1, strides = [1,1,1,1], padding = 'SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1],
                    strides = [1,2,2,1], padding ='SAME')
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)

In [7]:
#L2 (N, 14,14,32)
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
#    Conv      ->(N, 14, 14, 64)
#    Pool      ->(N, 7, 7, 64)
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1],
                    strides=[1, 2, 2, 1], padding='SAME')
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

In [8]:
#L2 (N, 7,7,64)
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
#    Conv      ->(N, 7, 7, 128)
#    Pool      ->(N, 4, 4, 128)
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1],
                    strides=[1, 2, 2, 1], padding='SAME')
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

In [9]:
#L3 (N,7,7,64)
W3 = tf.Variable(tf.random_normal([3,3,64,128], stddev = 0.01))
L3 = tf.nn.conv2d(L2, W3, strides=[1,1,1,1], padding = 'SAME')
L3 = tf.nn.max_pool(L3, ksize = [1,2,2,1], 
                    strides = [1,2,2,1], padding = 'SAME')
L3 = tf.nn.dropout(L3, keep_prob = keep_prob)
L3_flat = tf.reshape(L3, [-1,128*4*4])

In [10]:
#L4 4x4x128 > 625 outpus
W4 = tf.get_variable('W4', shape = [128*4*4, 625],
                    initializer = tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([625]))
L4 = tf.nn.relu(tf.matmul(L3_flat, W4) + b4)
L4 = tf.nn.dropout(L4, keep_prob = keep_prob)

In [11]:
#L5 FC 625 > 10 outputs
W5 = tf.get_variable("W5", shape = [625, 10],
                    initializer = tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([10]))
logits = tf.matmul(L4,W5) + b5

In [12]:
#defince loss function and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits= logits, labels = Y))
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)

#initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [13]:
#train model
print("Learning Start")
for epoch in range(training_epochs):
    avg_cost= 0
    total_batch = int(mnist.train.num_examples/ batch_size)
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X: batch_xs, Y: batch_ys, keep_prob: 0.7}
        c, _ = sess.run([cost, optimizer], feed_dict = feed_dict)
        avg_cost += c / total_batch
        
    print('Epoch : ', '%04d' % (epoch + 1), 'cost = ' ,'{:.9f}'.format(avg_cost))

print('Learinig Finished')

Learning Start
Epoch :  0001 cost =  0.242373252
Epoch :  0002 cost =  0.084420670
Epoch :  0003 cost =  0.067301359
Epoch :  0004 cost =  0.056301859
Epoch :  0005 cost =  0.051233445
Epoch :  0006 cost =  0.046876752
Epoch :  0007 cost =  0.043908921
Epoch :  0008 cost =  0.040029524
Epoch :  0009 cost =  0.039440567
Epoch :  0010 cost =  0.039329104
Epoch :  0011 cost =  0.034883739
Epoch :  0012 cost =  0.036451579
Learinig Finished


In [17]:
#Test moddle and check accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels, keep_prob: 1}))

Accuracy: 0.9923


In [18]:
# Get one and predict
r = random.randint(0, mnist.test.num_examples - 1)
print("Label: ", sess.run(tf.argmax(mnist.test.labels[r:r + 1], 1)))
print("Prediction: ", sess.run(tf.argmax(logits, 1), feed_dict={X: mnist.test.images[r:r + 1], keep_prob: 1}))

Label:  [7]
Prediction:  [7]
